In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import json

In [22]:
# import data
df_data = pd.read_csv('../data/ogd107_strom_import_export.csv', sep=',', decimal='.', thousands="'")
df_data.head()

,Datetime,AT_CH_MWh,DE_CH_MWh,FR_CH_MWh,IT_CH_MWh,CH_AT_MWh,CH_DE_MWh,CH_FR_MWh,CH_IT_MWh,Nettoimport
0,2017-01-01T00:00:00+0100,793,2953.0,0,1110,0,566.0,1451,0,2839.0
1,2017-01-01T01:00:00+0100,897,2969.0,0,1027,0,462.0,1174,0,3257.0
2,2017-01-01T02:00:00+0100,893,3557.0,0,963,0,621.0,1100,0,3692.0
3,2017-01-01T03:00:00+0100,981,3599.0,0,5,0,386.0,437,63,3699.0
4,2017-01-01T04:00:00+0100,1112,3859.0,0,0,0,442.0,391,519,3619.0


In [23]:
# fix date types
df_data['Datetime'] = pd.to_datetime(df_data['Datetime'], utc=True)
df_data['Year'] = df_data['Datetime'].dt.year

df_data.head()

,Datetime,AT_CH_MWh,DE_CH_MWh,FR_CH_MWh,IT_CH_MWh,CH_AT_MWh,CH_DE_MWh,CH_FR_MWh,CH_IT_MWh,Nettoimport,Year
0,2016-12-31 23:00:00+00:00,793,2953.0,0,1110,0,566.0,1451,0,2839.0,2016
1,2017-01-01 00:00:00+00:00,897,2969.0,0,1027,0,462.0,1174,0,3257.0,2017
2,2017-01-01 01:00:00+00:00,893,3557.0,0,963,0,621.0,1100,0,3692.0,2017
3,2017-01-01 02:00:00+00:00,981,3599.0,0,5,0,386.0,437,63,3699.0,2017
4,2017-01-01 03:00:00+00:00,1112,3859.0,0,0,0,442.0,391,519,3619.0,2017


In [24]:
# rename collumns: from_to
df_data.rename(columns={
    'Year': 'Year',
    'AT_CH_MWh': 'AT_CH',
    'DE_CH_MWh': 'DE_CH',
    'FR_CH_MWh': 'FR_CH',
    'IT_CH_MWh': 'IT_CH',
    'CH_AT_MWh': 'CH_AT',
    'CH_DE_MWh': 'CH_DE',
    'CH_FR_MWh': 'CH_FR',
    'CH_IT_MWh': 'CH_IT',
    'Nettoimport': 'Nettoimport'
}, inplace=True)

df_data.head()

,Datetime,AT_CH,DE_CH,FR_CH,IT_CH,CH_AT,CH_DE,CH_FR,CH_IT,Nettoimport,Year
0,2016-12-31 23:00:00+00:00,793,2953.0,0,1110,0,566.0,1451,0,2839.0,2016
1,2017-01-01 00:00:00+00:00,897,2969.0,0,1027,0,462.0,1174,0,3257.0,2017
2,2017-01-01 01:00:00+00:00,893,3557.0,0,963,0,621.0,1100,0,3692.0,2017
3,2017-01-01 02:00:00+00:00,981,3599.0,0,5,0,386.0,437,63,3699.0,2017
4,2017-01-01 03:00:00+00:00,1112,3859.0,0,0,0,442.0,391,519,3619.0,2017


In [25]:
# group by year
grouped_data = df_data.groupby('Year').sum(numeric_only=True).reset_index()
grouped_data.head()

,Year,AT_CH,DE_CH,FR_CH,IT_CH,CH_AT,CH_DE,CH_FR,CH_IT,Nettoimport
0,2016,793,2953.0,0,1110,0,566.0,1451,0,2839.0
1,2017,6560004,21356665.0,4580694,271224,68504,3591716.0,2212936,20493448,6401983.0
2,2018,4725310,17565206.0,6741582,121956,542650,4453076.0,1755146,21405976,997206.0
3,2019,4314870,14673007.0,7685864,121098,565365,6302281.0,2011339,21230301,-3314447.0
4,2020,3494983,11055211.0,8065086,861226,1043968,6467526.0,2314104,17883936,-4233028.0


In [26]:
# group data
yearly_data = {}

# Iterate over each row in the DataFrame
for index, row in grouped_data.iterrows():
    year = row['Year']
    exports = {col[3:].lower(): row[col] for col in row.index if col.startswith('CH_')}
    imports = {col[:-3].lower(): row[col] for col in row.index if col.endswith('_CH')}
    netto = {country: imports[country] - exports[country] for country in exports.keys()}
    yearly_data[year] = {
        'exports': exports,
        'imports': imports,
        'netto': netto,
    }

In [27]:
# Convert the dictionary to a list of dictionaries for Highcharts
highcharts_series = [{'name': year, 'data': data} for year, data in yearly_data.items()]

# write to file
with open('../src/data/import-export-eu.json', 'w') as json_file:
    json.dump(highcharts_series, json_file, indent=2)
